In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Dropdown, fixed
from IPython.display import display
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

In [2]:
# Path to go up one level from the current folder to the parent folder
parent_path = os.path.join(os.getcwd(), '..')

# Path to the target data folder
data_folder_path = os.path.join(parent_path, 'Data')

In [3]:
menu_df = pd.read_csv(os.path.join(data_folder_path, 'menu_analysis.csv'))
customers_df = pd.read_csv(os.path.join(data_folder_path, 'customers.csv'))
stores_df = pd.read_csv(os.path.join(data_folder_path, 'stores.csv'))
structure_df = pd.read_csv(os.path.join(data_folder_path, 'structure.csv'))
size_df = pd.read_csv(os.path.join(data_folder_path, 'order_size.csv'))
transactions_df = pd.read_csv(
        os.path.join(data_folder_path, "transactions.csv"),
        parse_dates=['Transaction_Date']  # Modify as needed if format issues arise
    )

### Modification 1

In [9]:
transactions_df = pd.merge(transactions_df, menu_df[['Item_Number', 'Price']], on = 'Item_Number')

### Modification 2

In [10]:
# Ensure column names are referenced correctly
transactions_df['Transaction_Time'] = transactions_df['Transaction_Time'].astype(str)
transactions_df['Transaction_Date'] = transactions_df['Transaction_Date'].astype(str)

# Combine Date and Time columns into one datetime column
transactions_df['DateTime'] = pd.to_datetime(transactions_df['Transaction_Date'] + ' ' + transactions_df['Transaction_Time'])

# Extract day of week and hour
transactions_df['DayOfWeek'] = transactions_df['DateTime'].dt.day_name()  # Using day_name() to get the day of the week as a name
transactions_df['Hour'] = transactions_df['DateTime'].dt.hour


In [11]:
transactions_df.head()

,Order_Number,Transaction_Date,Transaction_Time,Store_Number,Customer_ID,Item_Number,DateTime,DayOfWeek,Hour,Price
0,142292,2024-01-14,00:43:25,STR_7,CUST3706,Item_4,2024-01-14 00:43:25,Sunday,0,5.5
1,134538,2023-07-22,03:11:53,STR_10,CUST2840,Item_4,2023-07-22 03:11:53,Saturday,3,5.5
2,105529,2024-06-20,08:51:22,STR_8,CUST1036,Item_4,2024-06-20 08:51:22,Thursday,8,5.5
3,34280,2024-05-06,17:40:34,STR_5,CUST0994,Item_4,2024-05-06 17:40:34,Monday,17,5.5
4,118343,2023-11-07,00:32:02,STR_5,CUST2229,Item_4,2023-11-07 00:32:02,Tuesday,0,5.5


### Modification 3

In [12]:
# Group by day of week and hour, summing the total amount
hourly_sales = transactions_df.groupby(['DayOfWeek', 'Hour'])['Price'].sum().reset_index()

In [13]:
hourly_sales.head()

,DayOfWeek,Hour,Price
0,Friday,0,9743.0
1,Friday,1,8733.5
2,Friday,2,8959.5
3,Friday,3,9237.0
4,Friday,4,9635.5


### Modification 4

In [16]:
# Create a pivot table
pivot_table = hourly_sales.pivot(index='Hour', columns='DayOfWeek', values='Price')
# Fill NaN values with 0
pivot_table = pivot_table.fillna(0)

In [17]:
pivot_table

DayOfWeek,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
Hour,,,,,,,
0,9743.0,9516.5,8476.0,9284.5,9870.0,8796.0,9327.5
1,8733.5,9315.5,8810.5,9263.0,9050.0,9697.0,9566.5
2,8959.5,9877.5,9561.0,9531.5,9612.0,8525.5,9575.0
3,9237.0,8933.0,9295.0,9044.0,10196.0,9918.5,9085.5
4,9635.5,9304.5,8702.0,8413.0,9104.5,8495.0,8298.5
5,9205.5,9226.5,9551.0,9830.0,9013.5,9524.5,9784.5
6,9266.0,9261.0,9157.5,9722.5,9106.0,8532.5,8573.5
7,9695.0,9258.5,9575.0,9015.5,8661.0,9482.5,9152.0
8,10793.0,9388.5,9000.5,9418.5,9567.0,8764.5,9309.0


# 12. Hourly Sales Heatmap

In [18]:
# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=pivot_table.values,
    x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    y=pivot_table.index,
    colorscale='Viridis',
    hovertemplate='Day: %{x}<br>Hour: %{y}<br>Total Sales: $%{z:.2f}<extra></extra>'
))

# Update layout
fig.update_layout(
    title='Hourly Sales Heatmap',
    xaxis_title='Day of Week',
    yaxis_title='Hour of Day',
    yaxis = dict(
        tickmode = 'array',
        tickvals = list(range(24)),
        ticktext = [f'{i:02d}:00' for i in range(24)]
    )
)

# Add annotations for peak hours
peak_hour = pivot_table.mean(axis=1).idxmax()
peak_day = pivot_table.mean(axis=0).idxmax()

fig.add_annotation(
    x=peak_day,
    y=peak_hour,
    text="Peak Hour",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363"
)

# Add a color bar
fig.update_traces(colorbar=dict(
    title="Total Sales ($)",
    titleside="right",
    titlefont=dict(size=14),
    tickfont=dict(size=12),
))

fig.show()